# Fundamentals of machine learning
This section will formalize some new intuition into a solid conceptual framework for attacking and solving deep-learning problems. We will consilidate these concepts - model evaluation, data preprocessing and feature engineering, and tackling overfitting - into a detailed seven-step workflow for tackling any machine learning task.

## 4.1 Four branches of machine learning
### 4.1.1 Supervised learning
This is the most common case by far. It consists of learning to map input data to known targets, given a set of examples. Generally, almost all applications of deep learning that are in the spotlight these days belong in this category, such as optical character recognition, speech recognition, image classification, and language translation.

Although supervised learning consists mostly of classification and regression, there are more exotic variants as well, including the following:
 - **Sequence generation** - Given a picture, predict a caption describing it. Sequence generation can sometimes be reformulated as a series of classification problems.
 - **Syntax tree prediction** - Given a sentence, predict its decomposition into a syntax tree.
 - **Object detection** - Given a picture, draw a bounding box around certain objects inside the picture. This can also be expressed as a classification problem or as a joint classification and regression problem, where the bounding-box coordinates are predicted via vector regression.
 - **Image segmentation** - Given a picture, draw a pixel-level mask on a specific object.
 
### 4.1.2 Unsupervised learning
This branch of machine learning consists of finding interesting transformations of the input data without the help of any targets, for the purposes of data visualization, data compression, or data denoising, or to better understand the correlatons present in the data at hand. This is very common for data analytics, and it's often a necessary step in better understanding a dataset before attempting to solve a supervised learning problem. *Dimensionality reduction* and *clustering* are well-known categories of unsupervised learning.

### 4.1.3 Self-supervised learning
Self-supervised learning is supervised learning without human-annotated labels. There are still labels involved, but they are generated from the input data, typically using a heuristic algorithm.

For instance, *autoencoders* are a well-known instance of self-supervised learning, where the generated targets are the input, unmodified. Some instances include trying to predict the next frame in a video, given past frames, or the next word in a text, given previous words.

### 4.1.4 Reinforcement learning
In reinforcement learning, an *agent* receives information about its environment and learns to choose actions that will maximize some reward. For instance, a neural network that "looks" at a video game screen and outputs game actions in order to maximize its score can be trained via reinforcement learning. Some other examples include self-driving cars, robotics, resource management, education, and more.

## 4.2 Evaluating machine learning models
In machine learning, the goal is to achieve models that *generalize* - that perform well on never-before-seen data- and overfitting is the central obstacle. It's crucial to be able to reliably measure the generalization power of your model. We will look at strategies for mitigating overfitting and maximizing generalization.

### 4.2.1 Training, validation, and test sets
Evaluating a model always boils down to splitting the available data into three sets: **training, validation,** and **test**. The model is trained on the training data and evaluated on the validation data. Once the model is ready, it is tested a final time on the test data.

Why not only have a training and testing set? This is not the case because developing a model always involves tuning its configuration: choosing the number of layers, the size of layers (hyperparameters), number of epochs, and more. The validation data provides a feedback signal highlighting the model's performance. In fact, tuning the configuration of a model based on its performance on the validation set can lead to overfitting on the validation set, even though the model was not trained on it.

Every time you tune a a hyperparameter of your model based on the model's performance on the validation set, some information about the validation data leaks into the model. If you continue to tweak hyperparameters over and over again, you will end up with a model that performs artificially well on the validation data.

Splitting data into training, validation, and test sets may seem straightforward, but there are a few advanced ways to do it that can come in handy when faced with a sparse dataset. We will review three classic methods: simple hold-out validation, K-fold validation, and iterated K-fold validation with shuffling.

**SIMPLE HOLD-OUT VALIDATION**
Set apart some fraction of you data as your test set. Train on the remaining data, and evaluate on the test set. In order to prevent information leaks, you shouldn't tune your model based on the test set, and should also reserve a validation set.

![holdout](images/4_2_1_holdout.jpg)

Here is a simple code implementation of this:

In [ ]:
num_validation_samples = 10000

np.random.shuffle(data)

validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]

training_data = data[:]                                    

model = get_model()                                        
model.train(training_data)                                 
validation_score = model.evaluate(validation_data)         

# At this point you can tune your model,
# retrain it, evaluate it, tune it again...

model = get_model()                                        
model.train(np.concatenate([training_data,                 
                            validation_data]))             
test_score = model.evaluate(test_data)                     

This is the simplest evaluation protocol, but suffers from one flaw: if little data is available, then the validation and test sets may contain too few samples to be statistically representative of the data. This is easy to recognize: if different random shuffling rounds of the data before splitting end up yielding very different measures of model performance, then you're having this issue.

**K-FOLD VALIDATION**
With this approach, you split your data into *K* partitions of equal size. For each partition *i*, train a model on the remaining *K-1* partitions, and evaluated on partition *i*. The final score is the averager of the *K* scores obtained.

![threefold](images/4_2_1_threefold.jpg)

Here is a simple code implementation:

In [ ]:
k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold:
     num_validation_samples * (fold + 1)]
    training_data = data[:num_validation_samples * fold] +            
     data[num_validation_samples * (fold + 1):]                       
    model = get_model()                                               
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)

validation_score = np.average(validation_scores)                      

model = get_model()                                                   
model.train(data)                                                     
test_score = model.evaluate(test_data)                                

**ITERATED K-FOLD VALIDATION WITH SHUFFLING**
This should be used in situations where you have relatively little data available and you need to evaluate your model as precisely as possible. It consists of applying K-fold validation multiple times, shuffling the data every time before splitting it *K* ways. The final score is the average of the scores obtained at each run of K-fold validation. 

### 4.2.2 Things to keep in mind
Keep an eye out for the following when choosing an evaluation protocol:
 - **Data representatives** - You want your training and test sets to be representative of the data at hand. For example, if you're trying to classify images where the samples are ordered by their class and you take the first 80%. The other 20% can be a different class entirely that isn't even represented in the training set. For this reason, you should *randomly shuffle* the data before splitting it into training and testing sets.
 - **The arrow of time**- If you're trying to predict the future given the past (weather, stock movements, etc.), you should NOT randomly shuffle your data before splitting it. Doing so will create a *temporal leak*. The model will be effectively trained on data from the future. You should always make sure your test is *posterior* to the data in the training set.
 - **Redundancy in your data** - If some data points in your data appear twice, then shuffling the data and splitting it into a training set and validation set will result in redundancy between the training and validation sets. In essence, you will be testing on part of your training data, which is the worst thing you can do! 
 
## 4.3 Data preprocessing, feature engineering, and feature learning
### 4.3.1 Data preprocessing for neural networks
Data preprocessing aims at converting data to a proper format to feed into neural networks. It's sort of like feeding food to a child - you need to cut it up into small pieces and microwave it before it can be consumed. Data conversion includes vectorization, normalization, handling missing values, and feature extraction.

**VECTORIZATION**

All inputs and targets in a neural network must be tensors of floating-point data. Whatever data you need to process, you must first turn into tensors, a step called *data vectorization*. 

**VALUE NORMALIZATION**

In a digit-classification example, we started with image data encoded as integers between 0 and 255, encoded grayscale values. Before this data was fed into the network, we had to cast it to `float32` and divide by 255 so we'd end up with floating-point values between 0 and 1.

In general, it isn't safe to feed relatively large values of data into a neural network, or data that is heterogeneous. Doing so can trigger large gradient updates that will prevent the network from converging. To make learning easier for your network, your data should have the following characteristics:
 - **Take small values** - Typically, most values should be in the 0-1 range.
 - **Be homogenous** - All features should take values in roughly the same range.
 
Additionally, a stricter normalization practice is common and can help:
 - Normalize each feature independently to have a mean of 0.
 - Normalize each feature independently to have a standard deviation of 1.
 
This is easy to do with Numpy arrays:

In [ ]:
x -= x.mean(axis=0) # assuming x is a 2D matrix of shape (samples, features)
x /= x.std(axis=0)

**HANDLING MISSING VALUES**

Sometimes we may have to deal with missing values in our dataset. In general, with neural networks, it's safe to input missing values as 0, with the condition that 0 isn't already a meaningful value. The network will learn from exposure to the data that the value 0 means *missing data* and will start ignoring the value. Be careful when dealing with missing values in your test set. Make sure there are also some in your training set.

### 4.3.2 Feature engineering
*Feature engineering* is the process of using your own knowledge about the data and the algorithm at-hand to make the algorithm work better by applying hardcoded transformations to the data before it goes into the model.

For example, imagine a model that takes an image of a clock as its input and is expected to output the time of day.

![feature_eng](images/4_3_2_feature_eng.jpg)

If you choose to use the raw pixels of the images as input data, then you have a difficult machine learning problem on your hands. You'll need a convolutional neural network to solve it. You can come up with a much better input feature for a machine learning algorithm. For instance, it's easy to write a five-line Python script to follow the black pixels of the clock hands and output the (x,y) coordinates of the tip of each hand. Then a simple machine learning algorithm can learn to associate these coordinates with the appropriate time of day. 

The essence of feature engineering is making a problem easier by expressing it in a simpler way. Before deep learning, feature engineering used to be critical, because classical shallow algorithms didn't have hypothesis spaces rich enough to learn useful features by themselves. Fortunately, modern deep learning removes the need for most feature engineering, because neural networks are capable of automatically extracting useful features from raw data.

 - Good features still allow you to solve problems more elegantly while using fewer resources. 
 - Good features let you solve a problem with far less data.
 
## 4.4 Overfitting and underfitting
Overfitting happens in every machine learning problem. Learning how to deal with it is essential to mastering machine learning. The fundamental issue in machine learning is the tension between optimization and generalization. *Optimization* refers to the process of adjusting a model to get the best performance possible on the training data, whereas *generalization* refers to how well the trained model performs on data it has never seen before.

At the beginning of training, optimization and generalization are correlated. While this is happening, your model is said to be *underfit*. The network hasn't yet modeled all relevant patterns in the training data. But after a certain number of iterations on the training data, generalization stops improving, and validation metrics stall and then begin to degrade. The model is beginning to learn patterns that are specific to the training data but are misleading or irrelevant when it comes to new data, i.e. overfitting.

To prevent a model from overfitting, the best solution is to get more training data. A model trained on more data will naturally generalize better. The next best solution is to modulate the quantity of information the model is allowed to store. If a network can only afford to memorize a small number of patterns, the optimization process will force it to focus on the most significant patterns, which have a better chance of generalizing well.

The process of fighting overfitting this way is called *regularization*. Next, we will review some of the most common regularization techniques.

### 4.4.1 Reducing the network's size
The simplest way to prevent overfitting is to reduce the size of the model: the number of learnable parameters in the model (number of layers and number of units per layer). In deep learning, the number of learnable parameters in a model is often referred to as the model's *capacity*. A model with more parameters has more *memorization capacity* and can easily learn a perfect mapping between training samples and their targets, **but without any generalization power**. Deep learning models tend to be good at fitting to the training data, but the real challenge is generalization, not fitting.

Conversely, if the network has limited memorization resources, it won't be able to learn this mapping as easily; thus, in order to minimize its loss, it will have to resort to learning compressed representations that have predictive power regarding the targets - exactly the types of representations we are interested in! Keep in mind that your should use models that have enough parameters that they don't underfit. There is a compromise between *too much capacity* and *not enough capacity*.

There is no magical formula to determine the right number of layers or the right size for each layer. One must evaluate an array of different architectures (on your validation set, that is) in order to find the correct model size for your data. The general workflow to determine a proper model size is to start with relatively few layers and parameters, and increase the size of the layers or add new layers until you see diminishing returns with regard to validation loss. Let's try with the movie-review classification network:

In [2]:
# ORIGINAL MODEL
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [3]:
# Now replace it with a smaller network
model = models.Sequential()
model.add(layers.Dense(4, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

![smaller capacity](images/4_4_1_smallermodel.jpg)

Cool! What we can see here is that our original model starts with a lower validation loss during the first few epochs, but the smaller model is able to attain a much lower validation loss around epoch 6 and its loss doesn't incline as steeply as the original model as the number of epochs increases.

Now let's try adding a network that has much more capacity:

In [4]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

![bigger model](images/4_4_1_biggermodel.jpg)

Ouch! The bigger model starts overfitting almost immediately, and does so much more severely than our original model.

In addition, the plot below shows the training losses for the two networks. The bigger networks gets its training loss near zero very quickly. The added capacity enabled the bigger model to quickly fit to the training data, but it is much more seceptible to overfitting.

![capacity](images/4_4_1_capacity.jpg)

### 4.4.2 Adding weight regularization
Let's talk about *Occam's Razor*: given two explanations for something, the explanation most likely to be correct is the simplest one. This idea also applies to the models learned by neural networks: given some training data and a network architecture, multiple sets of weight values could explain the data. Simpler models are less likely to overfit than complex ones.
A common way to mitigate overfitting is to put constraints on the complexity of a network by forcing its weights to take only small values, which makes the distribution of the weight values more *regular*. This is called *weight regularization*, and its done by adding a cost associated with large weights to the loss function of the network. This cost comes in two variations:
 - **L1 Regularization**- The cost added is proportional to the *absolute value* of the weight coefficients (L1 norm)
 - **L2 Regularization**- The cost added is proportional to the *square of the value* of the weight coefficients. This is also called *weight decay* in the context of neural networks.
 
Here's how weight regularization is implemented in Keras:

In [6]:
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), 
                       activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

`l2(0.001)` means every coefficient in the weight matrix of the layer will add `0.001 * weight_coefficient_value` to the total loss of the network. The image below shows the impact of adding regularization to a model:

![L2](images/4_4_2_l2.jpg)

The model with L2 regularization (dots) has become much more resistant to overfitting than the original model, even though the models have the same number of parameters.

As an alternative to L2 regularization, you can use of the following Keras weight regularizers.

In [7]:
from keras import regularizers

regularizers.l1(0.001)

regularizers.l1_l2(l1=0.001, l2=0.001) # simultaneous L1 and L2 regularization

### Adding dropout
**Dropout** is one of the most effective and most commonly used regularization techniques for neural networks. Dropout, applied to a layer, consists of randomly *dropping out* (setting to zero) a number of output features of the layer during training. The dropout rate is the fraction of the features that are zeroed out; it's usually set between 0.2 and 0.5. At test time, no units are dropped out; instead, the layer's output values are scaled down by a factor equal to the dropout rate, to balance for the fact that more units are active than at training time.

Consider a Numpy matrix containing the output of a layer, `layer_output`, of shape (`batch_size, features`). At training time, we zero out a fraction of the values in the matrix at random:

In [ ]:
# at training time, this drops out 50% of the units in the output
layer_output *= np.random.randint(0, high=2, size=layer_output.shape)

At test time, we scale down the output by the dropout rate. Here, we scale by 0.5 (because we previously dropped half the units):

In [ ]:
# at test time
layer_output *= 0.5

This process can be implemented by doing both operations at training time and leaving the output unchanged at test time, which is often the way it's implemented in practice:

In [ ]:
layer_output *= layer_output *= np.random.randint(0, high=2, size=layer_output.shape)
layer_output /= 0.5 #note that we are scaling up, not down in this case

![dropout](images/4_4_3_dropout.jpg)

The core idea is that introducing noise in the output values of a layer can break up happenstance patterns that aren't significant, which the network will start memorizing if no noise is present.

Here's how you introduce a `Dropout` layer in Keras:

In [ ]:
model.add(layers.Dropout(0.5))

Let's add two `Dropout` layers to the IMDB network to see how well they do at reducing overfitting:

In [8]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

Here is a plot of the results. This shows a clear improvement over the original model.

![dropout](images/4_4_3_dropout2.jpg)

## 4.5 The universal workflow of machine learning
In this section, we will present a universal blueprint that you can use to attack and solve any machine learning problem.

### 4.5.1 Defining the problem and assembling a dataset
First comes defining the problem at hand:
 - What will your input data be? What are you trying to predict? You can only learn to predict something if you have available training data: for example, you can only learn to classify sentiment of movie reviews if you have both movie reviews and sentiment annotations available.
 - What type of problem are you facing? Is it binary classification? Multiclass classification? Scalar regression? Vector regression? Multiclass, multilabel classification? Something else, like clustering, generation, or reinforcement learning?
 
You can't move to the next stage until you know what your inputs and outputs are, and what data you'll use. At this stage you make the following hypotheses:
 - Your outputs can be predicted given your inputs.
 - Your available data is sufficiently informative to learn the relationship between inputs and outputs.
 
### 4.5.2 Choosing a measure of success
To control something, you need to be able to observe it. To achieve success, you must define what you mean by success--accuracy? Precision and recall? Customer-retention rate? You metric for success will guide the choice of a loss function: what your model will optimize. 

For balanced-classification problems, where every class if equally likely, accuracy and *area under the receiver operating characteristic curve (ROC AUC)* are common metrics. For class-imbalanced problems, you can use precision and recall. For ranking problems or multilabel classification, you can use mean average precision. You may end up using a custom metric as well!

### 4.5.3 Decising on an evaluation protocol
Once you know what you're aiming for, you must establish how you'll measure your current progress. Here are three common protocols:
 - **Maintaining a hold-out validation set** - use when you have plenty of data
 - **Doing K-fold cross-validation** - This is the right choice when you have too few samples for hold-out validation.
 - **Doing iterated K-fold validation**- For performing highly accurate model evaluation when little data is available.

### 4.5.4 Preparing your data
You need to format your data so it can be fed into a machine learning model, here we'll assume a neural network:
 - Data should be formatted as tensors
 - Usually scaled to small values: in the [-1,1] range or [0,1]
 - If different features take values in different ranges, then the data should be normalized.
 - Do feature engineering, especially for small data problems.
 
### 4.5.5 Developing a model that does better than a baseline
Your goal at this stage is to achieve statistical power: develop a small model that can beat a dumb baseline. If things go well, you need to make three key choices to build your first working model:
 - **Last layer activation**- example: sigmoid
 - **Loss function**- example: binary_crossentropy
 - **Optimization configuration**- example: rmsprop, mse
 
Note that it isn't always possible to directly optimize for success metric. A loss function needs to be computable given only a mini-batch of data, and must be differentiable.

Here is a useful table to help choose the right last-layer activation and loss function for your model:

![activation_loss_table](images/4_5_5_table.png)


### 4.5.6 Scaling up: developing a model that overfits
Once you have a model that has statistical power, you must begin to ask if the model is powerful enough. Does it have enough layers and parameters to properly model the problem at hand? The ideal model is one that stands right at the border between underfitting and overfitting; between undercapacity and overcapacity. To figure out how big a model you'll need, you must develop a model that overfits:
 1. Add layers
 2. Make layers bigger
 3. Train for more epochs
 
Always monitor the training loss and validation loss, as well as the training and validation values for any metrics you care about. Once the model's performance on the validation data begins to worsen, you've achieved overfitting.

The next stage is to start regularizing and tuning the model, to get as close as possible to the ideal model that neither underfits or overfits.

### 4.5.7 Regularizing your model and tuning your hyperparameters
This step generally takes the most time: you will repeatedly modify the model, train it, evaluate on validation data, modify again, and repeat until the model is as good as it can get. Here are some strategies to try:
 - Add dropout
 - Try different architectures: add or remove layers
 - Add L1 and/or L2 regularization
 - Try different hyperparameters (number of units per layer, optimizer learning rate, etc.)
 - Feature engineering
 
Be cautious of information leak every time you use feedback from your validation process. Once the model has reached a good configuration, you can train the final production model on all the available data (training and validation) and evaluate it one last time on the test set. If the performance on the test set was significantly worse than the performance on the validation set, this means the validation procedue wasn't reliable, or you began to overfit to the validation data while tuning parameters. If this is the case, you may want to switch to a more reliable evaluation protocol (such as iterated K-fold).